# 手动实现实现奇异值分解(SVD)用于计算伪逆矩阵

在常用的机器学习库 scikit-learn 中, 我们经常会使用 LinearRegression 来进行线性回归分析, 其函数内部使用的是 scipy.linalg.lstsq 函数，该函数使用了最小二乘法求解线性方程组。具体来说，它采用了 LAPACK 库中的 dgelsd 例程，该例程基于奇异值分解（SVD, Singular Value Decomposition），这是一个数值稳定的方法，适用于求解即使是奇异或接近奇异的矩阵。利用SVD，可以稳定地求解线性回归的解析解，即使矩阵是奇异的。
在这里，我们将手动实现奇异值分解(SVD)用于计算伪逆矩阵，以此来求解线性回归的解析解。

## 1. 奇异值分解 (SVD)

奇异值分解 (SVD) 将一个矩阵分解为三个矩阵的乘积：$\mathbf{A} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T$
其中：
- $\mathbf{A}$ 是一个 $m \times n$ 的矩阵。
- $\mathbf{U}$ 是一个 $m \times m$ 的正交矩阵。
- $\mathbf{\Sigma}$ 是一个 $m \times n$ 的对角矩阵，其对角线上的元素是矩阵 $\mathbf{A}$ 的奇异值。
- $\mathbf{V}^T$ 是一个 $n \times n$ 的正交矩阵的转置。

### 1.0 SVD 计算示例
假设我们有一个 $3 \times 2$ 的矩阵：$\mathbf{A} = \begin{pmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{pmatrix}$

为了详细求解矩阵 $\mathbf{A} = \begin{pmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{pmatrix}$ 的奇异值分解 (SVD)，即求 $\mathbf{A} = \mathbf{U} \mathbf{\Sigma} \mathbf{V}^T$，我们需要执行以下步骤：



### 1.1 步骤1：计算 $\mathbf{A}^T \mathbf{A}$ 和 $\mathbf{A} \mathbf{A}^T$
对于任何矩阵 $\mathbf{A}$，无论它是否是方阵，$\mathbf{A}^T \mathbf{A}$ 和 $\mathbf{A} \mathbf{A}^T$ 都一定是方阵。
这两个乘积结果方阵的尺寸分别取决于原矩阵的行数和列数。这种性质在许多数学和工程应用中非常有用，特别是在涉及到求解最小二乘问题和进行矩阵分解时。
$$
\mathbf{A}^T = \begin{pmatrix} 1 & 3 & 5 \\ 2 & 4 & 6 \end{pmatrix}
$$

$$
\mathbf{A}^T \mathbf{A} = \begin{pmatrix} 1 & 3 & 5 \\ 2 & 4 & 6 \end{pmatrix} \begin{pmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{pmatrix} = \begin{pmatrix} 35 & 44 \\ 44 & 56 \end{pmatrix}
$$

$$
\mathbf{A} \mathbf{A}^T = \begin{pmatrix} 1 & 2 \\ 3 & 4 \\ 5 & 6 \end{pmatrix} \begin{pmatrix} 1 & 3 & 5 \\ 2 & 4 & 6 \end{pmatrix} = \begin{pmatrix} 5 & 11 & 17 \\ 11 & 25 & 39 \\ 17 & 39 & 61 \end{pmatrix}
$$

### 1.2 步骤2：计算 $\mathbf{A}^T \mathbf{A}$ 的特征值和特征向量

特征值方程为：
$$
\det(\mathbf{A}^T \mathbf{A} - \lambda \mathbf{I}) = 0
$$
- 这里使用了行列式, 行列式$det()$是一个定义在方阵上的数学函数，其值可以从方阵中的元素通过特定的代数方法计算得到。行列式是方阵属性的核心概念之一，具有多种重要的数学和实际应用:
    - 解线性方程组：行列式用于判断线性方程组（表示为矩阵形式 $\mathbf{A}\mathbf{x} = \mathbf{b}$）是否有唯一解。当 $\text{det}(\mathbf{A}) \neq 0$ 时，方程组有唯一解。
    - 矩阵可逆性：行列式的值可以用来判断一个方阵是否可逆。具体来说，当且仅当 $\text{det}(\mathbf{A}) \neq 0$ 时，矩阵 $\mathbf{A}$ 是可逆的。
    - 几何解释：在几何学中，行列式可以解释为线性变换后形成的新图形与原图形的体积（或面积、长度）比例。例如，二维空间中一个矩阵的行列式表示对应线性变换下的面积扩大或缩小的倍数。
- 对于二阶和三阶行列式, 可以直接通过特定公式计算。例如，对于二阶行列式：$
  \text{det}\left(\begin{array}{cc}
  a & b \\
  c & d
  \end{array}\right) = ad - bc $
    - 对于更高阶的行列式，可以通过拉普拉斯展开沿某一行或某一列展开计算行列式。
- 这里, 特征值方程为：$\det(\mathbf{A}^T \mathbf{A} - \lambda \mathbf{I}) = 0$, 是为了找到使方阵 $\mathbf{A}^T \mathbf{A} - \lambda \mathbf{I}$ 成为奇异矩阵（即不可逆矩阵）的 $\lambda$值。
- 在代码中, 我们可以通过下面代码来实现行列式的计算：
```python
import numpy as np
from numba import njit, float64

@njit(float64(float64[:, :]))
def det_numba(matrix):
    """
    计算矩阵的行列式值。

    该函数使用Numba的Just-In-Time编译器优化，以提升数学运算的执行速度。
    它接受一个二维浮点数数组作为输入，并返回该矩阵的行列式值。

    参数:
    matrix: 二维浮点数数组，表示需要计算行列式的矩阵。

    返回值:
    浮点数，表示输入矩阵的行列式值。
    """
    # 使用NumPy的线性代数函数计算矩阵的行列式
    return np.linalg.det(matrix)
```


对于矩阵 $\mathbf{A}^T \mathbf{A}$：
$$
\mathbf{A}^T \mathbf{A} = \begin{pmatrix} 35 & 44 \\ 44 & 56 \end{pmatrix}
$$

其特征值方程为：
$$
\det\begin{pmatrix} 35 - \lambda & 44 \\ 44 & 56 - \lambda \end{pmatrix} = (35 - \lambda)(56 - \lambda) - 44^2 = 0
$$

展开并整理得到：
$$
35\times56 - 35\lambda - 56\lambda + \lambda^2 - 44^2 = 0
$$
$$
\lambda^2 - 91\lambda + 1960 - 1936 = 0
$$
$$
\lambda^2 - 91\lambda + 24 = 0
$$

解这个二次方程：
- 对于一般形式的二次方程：$a\lambda^2 + b\lambda + c = 0$, 其求解公式为: $\lambda = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$
- 因此:
$$
\lambda = \frac{91 \pm \sqrt{91^2 - 4 \cdot 24}}{2}
$$
$$
\lambda = \frac{91 \pm \sqrt{8281}}{2}
$$
$$
\lambda = \frac{91 \pm 90.9824}{2}
$$

计算得到两个特征值：
$$
\lambda_1 = \frac{91 + 90.9824}{2} = \frac{91+90.47098983}{2} = 90.73549491
$$
$$
\lambda_2 = \frac{91-90.47098983}{2} = \frac{0.0176}{2} = 0.264505087
$$

- 在python中, 我们可以通过下面代码来实现行列式的计算：
```python
import numpy as np
from numba import njit, float64

@njit(float64[:](float64[:, :]))
def compute_eigen_values(matrix):
    """
    计算给定矩阵的特征值。

    参数:
    matrix: float64[:, :] 类型的二维数组，表示输入的矩阵, 需要是方阵。

    返回值:
    float64[:] 类型的一维数组，包含输入矩阵的特征值。
    """
    # 使用numpy的linalg.eigvals函数计算矩阵的特征值
    eigen_values = np.linalg.eigvals(matrix)
    # 由于奇异值矩阵中奇异值的顺序是按降序排列的, 所以需要进行排序操作
    eigen_values[:] = np.sort(eigen_values)[::-1]  # 从大到小排序, 避免分配新的内存空间
    # 返回计算得到的特征值数组
    return eigen_values
```

### 1.3 步骤3：计算奇异值$\mathbf{\Sigma}$

我们已经得到了 $\mathbf{A}^T \mathbf{A}$ 的特征值：$\lambda_1 = 90.73549491$ 和 $\lambda_2 = 0.264505087$

奇异值是这些特征值的非负平方根：
$$\sigma_1 = \sqrt{\lambda_1} = \sqrt{90.73549491} \approx 9.524$$
$$\sigma_2 = \sqrt{\lambda_2} = \sqrt{0.264505087} \approx 0.514$$

因此，$\mathbf{\Sigma}$ 矩阵为：
$$
\mathbf{\Sigma} = \begin{pmatrix}
9.524 & 0 \\
0 & 0.514 \\
0 & 0
\end{pmatrix}
$$

- 在python中, 我们可以通过下面代码来实现行列式的计算：
```python
@njit(float64[:, :](float64[:], int64, int64))
def create_sigma_matrix(eigen_values, m, n):
    """
    根据给定的特征值数组，创建一个奇异值矩阵。

    参数:
    eigen_values: 一维数组，包含特征值。
    m: 整数，指定奇异值矩阵的行数。
    n: 整数，指定奇异值矩阵的列数。

    返回:
    一个二维数组，为生成的奇异值矩阵。
    """
    # 计算特征值的平方根，得到奇异值
    singular_values = np.sqrt(eigen_values)
    # 初始化一个大小为(m, n)的零矩阵, 初值为0
    the_sigma_matrix = np.zeros((m, n))
    # 将计算得到的奇异值填充到矩阵的对角线上
    np.fill_diagonal(the_sigma_matrix, singular_values[:min(m, n)])
    return the_sigma_matrix
```

### 1.4 步骤4：计算 $\mathbf{V}$ 矩阵

$\mathbf{V}$ 矩阵是 $\mathbf{A}^T \mathbf{A}$ 的特征向量矩阵。我们已经知道$\mathbf{A}^T \mathbf{A} = \begin{pmatrix} 35 & 44 \\ 44 & 56 \end{pmatrix}$的特征值为 $\lambda_1 = 90.73549491$, $\lambda_2 = 0.26450509$; 接下来就是找到特征向量 (Eigenvector)
- 特征向量是与特征值相关的向量。对于一个方阵 $\mathbf{A}$，如果存在一个非零向量 $\mathbf{v}$ 和一个标量 $\lambda$ 使得 $\mathbf{A} \mathbf{v} = \lambda \mathbf{v}$, 那么 $\mathbf{v}$ 就被称为矩阵 $\mathbf{A}$ 对应于特征值 $\lambda$ 的特征向量，$\lambda$ 被称为特征值。

- 在上述方程$\mathbf{A} \mathbf{v} = \lambda \mathbf{v}$中，矩阵 $\mathbf{A}$ 作用在向量 $\mathbf{v}$ 上时，结果仍然是向量 $\mathbf{v}$ 但被缩放了一个比例 $\lambda$。换句话说，矩阵 $\mathbf{A}$ 只改变了向量 $\mathbf{v}$ 的长度，而没有改变其方向。

- 基于$\mathbf{A} \mathbf{v} = \lambda \mathbf{v}$可以推导出$(\mathbf{A} - \lambda \mathbf{I}) \mathbf{v} = 0$, 将方阵A和特征值$\lambda$带入, 可以求出特征向量$\mathbf{v}$。

- 一般求解时, 会先假设一个特征向量的某个值为1，然后求解其他值，再进行L2范数的归一化处理。

- 对于每个特征值$\lambda$，都有一个对应的特征向量$\mathbf{v}$。如果有多个特征值, 可以构建出特征矩阵$\mathbf{V}$, 其中每一列是一个特征向量。


对于 $\lambda_1 = 90.73549491$，我们解以下方程：
$$
\begin{pmatrix} 35 - 90.73549491 & 44 \\ 44 & 56 - 90.73549491 \end{pmatrix} \begin{pmatrix} v_{11} \\ v_{21} \end{pmatrix} = \begin{pmatrix} -55.73549491 & 44 \\ 44 & -34.73549491 \end{pmatrix} \begin{pmatrix} v_{11} \\ v_{21} \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$
$$
\begin{pmatrix} -55.73549491 & 44 \\ 44 & -34.73549491 \end{pmatrix} \begin{pmatrix} v_{11} \\ v_{21} \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$
我们假设 $v_{11} = 1$，然后解得：
$$
-55.73549491 \cdot 1 + 44 \cdot v_{21} = 0 \implies v_{21} = \frac{55.73549491}{44} = 1.266
$$

解得特征向量 $\mathbf{v}_1$：
$$
\mathbf{v}_1 = \begin{pmatrix} 1 \\ 1.266 \end{pmatrix}
$$

归一化特征向量：
$$
\mathbf{v}_1 = \frac{1}{\sqrt{1^2 + 1.266^2}} \begin{pmatrix} 1 \\ 1.266 \end{pmatrix} = \frac{1}{1.598} \begin{pmatrix} 1 \\ 1.266 \end{pmatrix} = \begin{pmatrix} 0.626 \\ 0.793 \end{pmatrix}
$$

所以 $\mathbf{v}_1 = \begin{pmatrix} 0.626 \\ 0.793 \end{pmatrix}$。

对于 $\lambda_2 = 0.264505087$，我们解以下方程：
$$
\begin{pmatrix} 35 - 0.26450509 & 44 \\ 44 & 56 - 0.26450509 \end{pmatrix} \begin{pmatrix} v_{12} \\ v_{22} \end{pmatrix} = \begin{pmatrix} 34.73549491 & 44 \\ 44 & 55.73549491 \end{pmatrix} \begin{pmatrix} v_{12} \\ v_{22} \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$
$$
\begin{pmatrix} 34.73549491 & 44 \\ 44 & 55.73549491 \end{pmatrix} \begin{pmatrix} v_{12} \\ v_{22} \end{pmatrix} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}
$$

我们假设 $v_{12} = 1$，然后解得：
$$
34.73549491 \cdot 1 + 44 \cdot v_{22} = 0 \implies v_{22} = -\frac{34.73549491}{44} = -0.79
$$

解得特征向量 $\mathbf{v}_2$：
$$
\mathbf{v}_2 = \begin{pmatrix} 1 \\ -0.79 \end{pmatrix}
$$

归一化特征向量：
$$
\mathbf{v}_2 = \frac{1}{\sqrt{1^2 + (-0.79)^2}} \begin{pmatrix} 1 \\ -0.79 \end{pmatrix} = \frac{1}{1.265} \begin{pmatrix} 1 \\ -0.79 \end{pmatrix} = \begin{pmatrix} 0.79 \\ -0.625 \end{pmatrix}
$$

所以 $\mathbf{v}_2 = \begin{pmatrix} 0.79 \\ -0.625 \end{pmatrix}$。

最终求得特征矩阵为:
$$
\mathbf{V} = \begin{pmatrix} 0.626 & 0.793 \\ 0.79 & -0.625 \end{pmatrix}
$$

### 1.5 步骤5：计算 $\mathbf{U}$ 矩阵

$\mathbf{U}$ 矩阵是 $\mathbf{A} \mathbf{A}^T$ 的特征向量矩阵。我们计算出$\mathbf{A} \mathbf{A}^T = \begin{pmatrix} 5 & 11 & 17 \\ 11 & 25 & 39 \\ 17 & 39 & 61 \end{pmatrix}$
重复上面的计算过程, 可以求得$\mathbf{U}$ 矩阵。

### 1.6 代码实现

其实说了那么多, 我们可以通过 numpy.linalg 的 svd 函数来实现奇异值分解(SVD)。下面是一个简单的示例代码：
```python
import numpy as np

# 创建一个矩阵
A = np.array([[1, 2], [3, 4], [5, 6]])
U, S, VT = np.linalg.svd(A)
print("U:", U)
'''
[[-0.2298477   0.88346102  0.40824829]
 [-0.52474482  0.24078249 -0.81649658]
 [-0.81964194 -0.40189603  0.40824829]]
'''

print("S:", S)
'''
[9.52551809 0.51430058]
'''

print("VT:", VT)
'''
[[-0.61962948 -0.78489445]
 [-0.78489445  0.61962948]]
'''
```

## 2. 伪逆矩阵

伪逆矩阵（Pseudoinverse）是对矩阵求逆的一种广义形式。对于一个给定的矩阵 $\mathbf{A}$，其伪逆矩阵通常记作 $\mathbf{A}^+$。伪逆矩阵在某些情况下代替了逆矩阵，特别是在矩阵不可逆或不是方阵的情况下。
伪逆矩阵有多种定义，其中最常见的一种是使用奇异值分解（SVD）来计算的。

- Moore-Penrose 伪逆是最常用的一种伪逆，满足以下四个条件：
    1. $\mathbf{A} \mathbf{A}^+ \mathbf{A} = \mathbf{A}$
    2. $\mathbf{A}^+ \mathbf{A} \mathbf{A}^+ = \mathbf{A}^+$
    3. $(\mathbf{A} \mathbf{A}^+)^T = \mathbf{A} \mathbf{A}^+$
    4. $(\mathbf{A}^+ \mathbf{A})^T = \mathbf{A}^+ \mathbf{A}$
- 伪逆矩阵（Moore-Penrose Pseudoinverse）$\mathbf{A}^+$ 的计算公式为：$\mathbf{A}^+ = \mathbf{V} \mathbf{\Sigma}^+ \mathbf{U}^T$
    - 其中，$\mathbf{\Sigma}^+$ 是通过将 $\mathbf{\Sigma}$ 的非零奇异值取倒数，并在其他位置补零得到的。

### 2.1 计算奇异值矩阵的伪逆 $\mathbf{\Sigma}^+$

在第一部的奇异值分解中, 我们已经计算得到：
$$
\mathbf{\Sigma} = \begin{pmatrix}
9.52551809 & 0 \\
0 & 0.51430058 \\
0 & 0
\end{pmatrix}
$$

伪逆矩阵 $\mathbf{\Sigma}^+$ 是通过取倒数非零奇异值并转置得到的：
$$
\mathbf{\Sigma}^+ = \begin{pmatrix}
\frac{1}{9.52551809} & 0 & 0 \\
0 & \frac{1}{0.51430058} & 0
\end{pmatrix}
$$

计算得到：
$$
\mathbf{\Sigma}^+ = \begin{pmatrix}
0.10497737 & 0 & 0 \\
0 & 1.94438824 & 0
\end{pmatrix}
$$


### 2.2 计算伪逆矩阵 $\mathbf{A}^+$

根据伪逆矩阵的定义：
$$
\mathbf{A}^+ = \mathbf{V} \mathbf{\Sigma}^+ \mathbf{U}^T
$$

我们已经有：
$$
\mathbf{V} = \begin{pmatrix}
-0.61962948 & -0.78489445 \\
-0.78489445 & 0.61962948
\end{pmatrix}
$$
$$
\mathbf{U} = \begin{pmatrix}
-0.2298477 & 0.88346102 & 0.40824829 \\
-0.52474482 & 0.24078249 & -0.81649658 \\
-0.81964194 & -0.40189603 & 0.40824829
\end{pmatrix}
$$
$$
\mathbf{\Sigma}^+ = \begin{pmatrix}
0.10497737 & 0 & 0 \\
0 & 1.94438824 & 0
\end{pmatrix}
$$

首先计算 $\mathbf{V} \mathbf{\Sigma}^+$：
$$
\mathbf{V} \mathbf{\Sigma}^+ = \begin{pmatrix}
-0.61962948 & -0.78489445 \\
-0.78489445 & 0.61962948
\end{pmatrix} \begin{pmatrix}
0.10497737 & 0 & 0 \\
0 & 1.94438824 & 0
\end{pmatrix} = \begin{pmatrix}
-0.06507094 & -1.52597303 & 0 \\
-0.15300144 & 1.20450816 & 0
\end{pmatrix}
$$

然后计算 $\mathbf{A}^+$：
$$
\mathbf{A}^+ = \begin{pmatrix}
-0.06507094 & -1.52597303 \\
-0.15300144 & 1.20450816
\end{pmatrix} \mathbf{U}^T
$$
$$
\mathbf{U}^T = \begin{pmatrix}
-0.2298477 & -0.52474482 & -0.81964194 \\
0.88346102 & 0.24078249 & -0.40189603 \\
0.40824829 & -0.81649658 & 0.40824829
\end{pmatrix}^T = \begin{pmatrix}
-0.2298477 & 0.88346102 & 0.40824829 \\
-0.52474482 & 0.24078249 & -0.81649658 \\
-0.81964194 & -0.40189603 & 0.40824829
\end{pmatrix}
$$
$$
\mathbf{A}^+ = \begin{pmatrix}
-0.06507094 & -1.52597303 \\
-0.15300144 & 1.20450816
\end{pmatrix} \begin{pmatrix}
-0.2298477 & 0.88346102 & 0.40824829 \\
-0.52474482 & 0.24078249 & -0.81649658 \\
-0.81964194 & -0.40189603 & 0.40824829
\end{pmatrix}
$$
$$
\mathbf{A}^+ = \begin{pmatrix}
-0.96551724 & -0.03448276 & 0.89655172 \\
0.76428571 & 0.02857143 & -0.71428571
\end{pmatrix}
$$